## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-14-20-52-14 +0000,nyt,Newsom Held an Event in L.A. Federal Immigrati...,https://www.nytimes.com/2025/08/14/us/newsom-l...
1,2025-08-14-20-51-15 +0000,nypost,Warren Buffett’s Berkshire announces surprise ...,https://nypost.com/2025/08/14/business/warren-...
2,2025-08-14-20-43-25 +0000,startribune,Remains found in car at bottom of Mississippi ...,https://www.startribune.com/dive-team-discover...
3,2025-08-14-20-43-00 +0000,wsj,Warren Buffett’s Berkshire Hathaway Further Pa...,https://www.wsj.com/finance/stocks/berkshire-h...
4,2025-08-14-20-41-57 +0000,nypost,Ex-Goldman banker blew $4 million of investor ...,https://nypost.com/2025/08/14/business/ex-gold...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2327/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
63,trump,54
163,putin,21
85,new,14
6,up,12
57,court,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
171,2025-08-14-11-58-04 +0000,nypost,Russia appears ready to test new missile — eve...,https://nypost.com/2025/08/14/world-news/russi...,124
119,2025-08-14-15-59-01 +0000,nypost,Trump reveals the odds of Putin summit ending ...,https://nypost.com/2025/08/14/us-news/trump-gi...,115
69,2025-08-14-17-56-02 +0000,nypost,India shops the globe for oil after 50% tariff...,https://nypost.com/2025/08/14/world-news/india...,109
44,2025-08-14-19-03-21 +0000,cbc,"Zelenkyy, Starmer meet ahead of Trump, Putin s...",https://www.cbc.ca/player/play/9.6867280?cmp=rss,105
39,2025-08-14-19-27-10 +0000,nyt,Russia Seeks to Add Trade and Arms Control Int...,https://www.nytimes.com/2025/08/14/world/europ...,105


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
171,124,2025-08-14-11-58-04 +0000,nypost,Russia appears ready to test new missile — eve...,https://nypost.com/2025/08/14/world-news/russi...
30,48,2025-08-14-20-08-00 +0000,wsj,California Democrats are planning their own ea...,https://www.wsj.com/politics/policy/gavin-news...
100,41,2025-08-14-16-29-11 +0000,nypost,DC cops allowed to alert ICE about illegal imm...,https://nypost.com/2025/08/14/us-news/dc-cops-...
59,39,2025-08-14-18-15-15 +0000,nypost,Air Canada warns it will cancel 500 flights by...,https://nypost.com/2025/08/14/business/air-can...
31,35,2025-08-14-20-07-37 +0000,nypost,Mom of 2 fatally shot walking her kid to schoo...,https://nypost.com/2025/08/14/us-news/kentucky...
69,34,2025-08-14-17-56-02 +0000,nypost,India shops the globe for oil after 50% tariff...,https://nypost.com/2025/08/14/world-news/india...
153,34,2025-08-14-13-45-26 +0000,cbc,More than 100 aid groups call on Israel to end...,https://www.cbc.ca/news/world/israel-gaza-aid-...
174,30,2025-08-14-11-39-23 +0000,wapo,"Top South Korean court listens to ‘Baby Shark,...",https://www.washingtonpost.com/style/2025/08/1...
103,30,2025-08-14-16-23-09 +0000,nypost,"Toronto film festival reverses course, will ‘e...",https://nypost.com/2025/08/14/world-news/tiff-...
22,29,2025-08-14-20-16-09 +0000,nyt,Federal Prosecutors Seek Death Penalty in Fata...,https://www.nytimes.com/2025/08/14/us/border-p...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
